<a href="https://colab.research.google.com/github/LuanEverton/Project-CHATBOT-Amazon/blob/main/codigo_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código CHATBOT Amazon

In [ ]:
#!pip install openai twilio python-dotenv
#!pip install requests



In [ ]:
import os #biblioteca os utilizada para acessar compornentes do sistema


#aqui armazenamos as credenciais com o os.environ para que possa ser acessada com o getenv() depois sem que as credenciais sejam expostas.
#nesse caso o os.environ é necessário por ser criado no colab
os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_OPENAI"
os.environ["TWILIO_SID"] = "SEU_TWILIO_SID"
os.environ["TWILIO_AUTH_TOKEN"] = "SEU_TWILIO_AUTH_TOKEN"
os.environ["TWILIO_WHATSAPP_NUMBER"] = "whatsapp:+14155238886"
os.environ["WHATSAPP_GROUP_NUMBER"] = "whatsapp:+SEU_NUMERO"


In [ ]:
#bibliotecas utilzadas no chatbot

import requests
import hashlib
import hmac
import time
import base64

In [ ]:
#configurar as credeciais da API AMAZON

AWS_ACESS_KEY = os.getenv("AWS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
ASSOCIATE_TAG = os.getenv("ASSOCIATE_TAG")
REGION = "br-sul-1"
ENDPOINT = "websevice.amazon.com"

